# Исследование надежности заемщиков.

Заказчиком исследования выступает кредитный отдел банка. **Целью исследования является ответы на вопросы:**
1. Имеется ли корреляция между наличием у кредитора детей ( и их кол-ва ) и своевременной выплатой кредита?
2. Как влияет семейное положение кредитора на выплату кредитной задолжности в указанные сроки?
3. Имеет ли роль уровня дохода заёмщика на своевременные им выплаты по кредиту?
4. Есть ли взаимосвязь между целью взятия кредита и шансом его невыплаты?

Данные, полученные в ходе исследования и их интерпретация в конечных выводах будут использоваться заказчиком для построения *кредитного скоркинга*.

**Ход исследования**

Исследования и выводы на его основе будут проводиться по полученным статистическим данных кредиторов банка из файла `data.csv`. О качестве данных заведомо ничего неизвестно, поэтому предварительно необходим обзор данных, с оценкой на наличие и критичность имеющихся ошибок с последующей, по необходимости, предобработкой данных.

Таким образом исследование будет выполнено в 3 этапа:
1. Обзор данных
2. Предобработка данных
3. Ответ на поставленные вопросы и общий вывод


## Обзор данных

In [1]:
import pandas as pd # импорт библиотеки pandas

In [2]:
df = pd.read_csv('/datasets/data.csv') # выгрузка и сохранение датасета в переменную df

In [3]:
df.head(10) # получение первых 10 строк датасета

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
df.info() # получение общей информации о данных в таблице df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Согласно приложенной к данным документации, мы имеем 12 столбцов с данными:
* `cildren` - количество детей ( 0 в случае отсутствия);
* `days_employed` - общий трудовой стаж в днях;
* `dop_years` - возраст клиента в годах;
* `education` - уровень образования клиента;
* `education_id` - индификатор уровня образования клиентв;
* `family_status` - семейный статус клиента;
* `family_status_id` - индификатор семейного статуса клиента;
* `gender` - пол клиента;
* `income_type` - тип занятости клиента;
* `debt` - имел ли задожности по возврату кредита;
* `total_income` - ежемесячный доход;
* `purpose` - цель получения кредита.

Количество значений во всех столбцах, кроме `days_employed` и `total_income`, совпадает. 

**Выводы**

В каждой строке таблицы - данные о клиенте банка. Вся таблица содержит данные о клиенте, необходимые при учете выдачи кредита.

Предварительно можно утверждать, что данных достаточно для анализа. Однако, в таблице имеются ошибки и аномалии: отрицательные доходы, различие регистра в строковых столбцах, вещественные числа. Помимо этого таблица имеет большой размер, что снижает читаемость и информативность данных.

Что бы иметь возможность качественного анализа, необходимо устранить все ошибки и произвести обработку таблицы для лучшей читаемости.

## Предобработка данных

### Заполнение пропусков

В первую очередь посчитаем количество пропусков и узнаем, в каких слобцах они находятся:

In [5]:
df.isna().sum() # подсчет пропусков

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Как ранее было описано на предыдущем этапе, датасет имеет пропуски лишь в двух количественных столбцах `days_employed` и `total_income`. Так же можно заметить, что в обоих столбцах одинаковое количество пропусков, из-за чего можно предположить, что пропуски имеются в одних и тех же строках.

Далее, с помощью методов *pandas* узнаем тип пропусков и проверим теорию:

In [6]:
df[df['days_employed'].isna()].head(10) # вывод первых 10 строк с пропусками в столбце 'days_employed'

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Поскольку пропущенные данные имеют влияние на исследование, вычислем их процентное количество в датасете:

In [7]:
print('Процент строк с пропущенными значениями в таблице: {:.2%}'.format( # вычисление процента пропущенных строк через отношение строк с пропусками
    df['days_employed'].isna().sum() / df['children'].count()))           # к количеству значений в столбце без пропусков ( т.е. общему кол-ву строк)

Процент строк с пропущенными значениями в таблице: 10.10%


Не смотря на то, что `10%` пропусков является дозволительным количеством пропусков, эти пропускими находятся в столбцах, учавствующих в отдельно взятом исследование ( взаимосвязь между уровнем дохода и шансом невозврата кредита ). По этой причине запомним пропуски медианными значениями датасета. 

Медианные значения берутся по причине высокой устройчивости к аномальным значениям в таблице и минимизируют шанс искажения исследовательских данных.

In [8]:
df['days_employed'] = df['days_employed'].fillna( # заполнение столбца 'days_employed' медианными значениями в нём
    df['days_employed'].median()
) 
df['total_income'] = df['total_income'].fillna(   # заполнение столбца 'total_income' медианными значениями в нём
    df['total_income'].median()
) 

Убедимся, что все пропуски в столбцах датасета устранены:

In [9]:
df.isna().sum() # провеска кол-ва пропуска в таблице

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод**

По итогу мы заменили все пропуски в столбцах медианными значениями, исключив ошибки и искажения исследования в дальнейшем.
Учитывая привязанность обоих пропусков друг к другу, т.е. если в строке отсутствует стаж, то в ней так же отсутсвует и доход, то можно предположить, что при заполнении данных, информация о доходе и трудовом стаже клиента отсутвует, т.к. он является безработным.
Помимо этого, не стоит исключать ошибки со стороны разработчиков, которые могут привести к незаполнению значений.

### Проверка данных на аномалии и исправления.

На основе первых 10 строк датасета можно увидеть 2 основные аномалии в столбце `days_employed`:
* Отрицательные значения трудового стажа в днях;
* Трудовой стаж превышает возраст клиента.

Вычислим общее количество строк с отрицательными значениями:

In [10]:
print('Количество строк с отрицательным трудовым стажем:',     # через логическую индексацию подсчитываем общее количество
     df.loc[df['days_employed'] < 0]['days_employed'].count()) # отрицательных значений в столбце 'days_employed'

Количество строк с отрицательным трудовым стажем: 18080


Как мы видим, в столбце датасета большое количество аномалий, которые могут повлиять на результаты исследования. По этой причине исправим все значения.

Обработаем отрицательные значения, заменив их на свой же модуль:

In [11]:
# создаем цикл, который проходится по индексам датасета
for index in range(len(df)):
# если значение стажа меньше нуля, то:
    if df.loc[index, 'days_employed'] < 0:
# заменить это значение на свой же модуль
        df.loc[index, 'days_employed'] = abs(df.loc[index, 'days_employed'])

Так же по первым 10 строкам датасета можно заметить, что в том же столбце имеется аномально большой трудовой стаж, вычислим количество строк, где трудовой стаж превышает возвраст клиента в днях:

In [12]:
total_abnormal_seniority = 0 
# создаём цикл, который будет проходится по всем индексам таблицы
for index in range(len(df)):
# с помощью логической индексации, сравниваем трудовой стаж и возраст в днях одного клиента
    if df.loc[index, 'days_employed'] > ( df.loc[index, 'dob_years'] * 30 * 12):
# если стаж больше возвраста клиента, обновляем счётчик
        total_abnormal_seniority += 1
print('Количество строк с аномально большим стажем:', total_abnormal_seniority)

Количество строк с аномально большим стажем: 3529


Заменим аномально большие значения на медианные значения столбца:

In [13]:
# создаем цикл, который пройдётся по всем индексам датасета
for index in range(len(df)):
# если трудовой стаж ( в днях) превышает возраст клиента ( в днях), то:
    if df.loc[index, 'days_employed'] > ( df.loc[index, 'dob_years'] * 30 * 12):
# заменить текущее значение трудового стажа на медианное значение
        df.loc[index, 'days_employed'] = df['days_employed'].median()

Помимо видимых в первых строках аномалий, необходимо проверить на их наличие столбцы, которые будут в дальнейшем учавствовать в исследовании, а именно: столбцы кол-ва детей и уровня дохода.

Выведем список уникальных значений столбца `children` и подсчитаем количество отрицательных значений в столбце `total_income`:

In [14]:
# подсчёт количества отрицательных значений в столбце 'total_income'
print('Количество отрицательных значений дохода в таблице:',
     df.loc[df['total_income'] < 0]['total_income'].count())

df['children'].value_counts()

Количество отрицательных значений дохода в таблице: 0


 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Как можно заметить, столбец `total_income` не имеет аномальных значений. Однако в столбце `children` присутствуют оба, знакомые нам, вида аномалий: *отрицательные* и *аномально высокие* значения. 

Обработаем аномалии в столбце привычным нам образом, заменив их медианным значением в столбце:

In [15]:
# создаем цикл, который пройдётся по индексам таблицы
for index in range(len(df)):
# если значение о кол-ве детей равно 20, то:
    if df.loc[index, 'children'] == 20:
# заменить это значение на медианное
        df.loc[index, 'children'] = df['children'].median()
# если значение о кол-ве детей меньше 0, то:
    if df.loc[index, 'children'] < 0:
# заменить это значение на медианное:
        df.loc[index, 'children'] = df['children'].median()

Утостоверимся, что все аномалии в столбце `children` устранены. Вновь выведем уникальные значения этого столбца:

In [16]:
df['children'].value_counts()

0.0    14272
1.0     4818
2.0     2055
3.0      330
4.0       41
5.0        9
Name: children, dtype: int64

Последним столбцом для проверки аномалий будет `dob_years`, поскольку он так же имеет важное значение в исследовании.
Подсчитаем кол-во повторений каждого уникального значения в столбце:

In [17]:
df['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

Как можно заметить, и этот столбец имеет аномалию - у части клиентов возраст имеет нулевое значение. Заменим их на медианный показатель в этом столбце:

In [18]:
# создаем цикл, который пройдётся по всем индексам датасета
for index in range(len(df)):
# если возраст равен нулю, то:
    if df.loc[index, 'dob_years'] == 0:
# заменить текущий возраст на медианное значение
        df.loc[index, 'dob_years'] = df['dob_years'].median()

Удостоверимся, что все аномальные значения в столбце `dob_years` устранены:

In [19]:
df['dob_years'].value_counts()

42.0    698
35.0    617
40.0    609
41.0    607
34.0    603
38.0    598
33.0    581
39.0    573
31.0    560
36.0    555
44.0    547
29.0    545
30.0    540
48.0    538
37.0    537
50.0    514
43.0    513
32.0    510
49.0    508
28.0    503
45.0    497
27.0    493
56.0    487
52.0    484
47.0    480
54.0    479
46.0    475
58.0    461
57.0    460
53.0    459
51.0    448
59.0    444
55.0    443
26.0    408
60.0    377
25.0    357
61.0    355
62.0    352
63.0    269
64.0    265
24.0    264
23.0    254
65.0    194
66.0    183
22.0    183
67.0    167
21.0    111
68.0     99
69.0     85
70.0     65
71.0     58
20.0     51
72.0     33
19.0     14
73.0      8
74.0      6
75.0      1
Name: dob_years, dtype: int64

**Вывод**

Как можно заметить, мы обработали все явные и неявные аномалии в столбцах. В столбце `days_employed` аномалия с отрицательным значением стажа имеет массовый характер, что может говорить о системной ошибки. Все остальные аномалии могут иметь разносторонний характер. Поэтому сложно дать примерную и тем более однозначную причину образованных ошибок. Данные об этом следует передать разработчикам для дальнейшего решения проблем.

Помимо этого, после обработки ошибок, количественные данные приняли вид вещественных чисел, что осложняет читаемость данных. В следующем шаге обработаем данные, изменив их должным образом.

### Изменение типов данных.

После обработки аномалий, ряд столбцов принял вещественный тип данных. Вызовем информацию о таблице, что бы узнать, какие столбцы требуют изменения типа данных и убедиться, что отсутствуют столбцы численных значений со строчным типом данных.

In [20]:
df.info() # вызываем общую информацию о таблице

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null float64
days_employed       21525 non-null float64
dob_years           21525 non-null float64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(4), int64(3), object(5)
memory usage: 2.0+ MB


Как можем заметить, проблема с таблицией по части типизации данных заключается лишь в записи значений части столбцов в вещественный, а не целочисленный тип данных. 
Создадим функцию для обработки этих столбцов:

In [21]:
# создаём функцию <replace_data_type>:
# на вход поступает название столбца, который необходимо перезаписать
def replace_data_type(column):
# заменяем тип данных указанного столбца на int
    df[column] = df[column].astype('int')

In [22]:
# объявляем список с наименованиями столбцов, требующих обработки
columns = ['children', 'days_employed', 'dob_years', 'total_income']
# создаём цкл для перебора значений списка
for column in columns:
# вызываем функцию <replace_data_type> для столбца
    replace_data_type(column)

Удостоверимся, что тип данных во всех вызванных столбцах заменён:

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Удаление дубликатов.

На первых 10 строках датасета мы могли заметить неявные дубликаты, образованные из-за разного регистра значений. 
Напишим функцию, которая будет приводить все значения столбца к общему регистру:

In [24]:
# создаем функцию <replace_lower_str>:
# на вход поступает название столбца, который необходимо перезаписать
def replace_lower_str(column):
# приводим значения столбца к общему регистру, перезаписываем его
    df[column] = df[column].str.lower()

In [25]:
# объявляем список с названием столбцов таблицы, содержащие строчные значения
columns = ['education', 'family_status', 'gender', 'income_type', 'purpose']

# вызываем цикл, который переберёт все значения в списке columns
for column in columns:
# вызываем функцию <replace_lower_str> для столбца
    replace_lower_str(column)

Убедимся на примере столбца `education`, что функция и цикл сработали корректно, приведя значения к общему регистру:

In [26]:
df['education'].value_counts() # подсчет уникальных значений в столбце 'education'

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Посчитаем общее количество явных дубликатов в таблице:

In [27]:
df.duplicated().sum() # подсчет явных дубликатов в таблице

71

Удалим все явные дубликаты с последующей перезаписью индексов:

In [28]:
df = df.drop_duplicates().reset_index(drop = True) # удаление явных дубликатов (с удалением старых индексов и формированием новых)

Убедимся, что все явные дубликаты удалены:

In [29]:
df.duplicated().sum() # подсчет явных дубликатов в таблице

0

### Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Займёмся декомпозицией таблицы. Для удобного обращения с ней, создадим два "словаря":
* Первый словарь образуется из столбцов `education` и `education_id`
* Второй словарь образуется из столбцов `family_status` и `family_status_id`

In [30]:
family_status_dict = df[['family_status', 'family_status_id']] # создаём 'словарь' с индификатором статусов
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop = True) # удаляем все явные дубликаты
display(family_status_dict)

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


In [31]:
education_dict = df[['education', 'education_id']] # создаём 'словарь' с индификатором статусов
education_dict = education_dict.drop_duplicates().reset_index(drop = True) # удаляем все явные дубликаты
display(education_dict)

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


Удалим столбцы `education` и `family_status` из основного датасета:

In [32]:
df = df.drop('education', 1)     # удаляем из основного датасета столбец 'education'
df = df.drop('family_status', 1) # удаляем из основного датасета столбец 'family_status'

# выведем первые 10 строк обновленного датасета
df.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,f,сотрудник,0,253875,покупка жилья
1,1,4024,36,1,0,f,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,1,0,m,сотрудник,0,145885,покупка жилья
3,3,4124,32,1,0,m,сотрудник,0,267628,дополнительное образование
4,0,1808,53,1,1,f,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,0,1,m,компаньон,0,255763,покупка жилья
6,0,2879,43,0,0,f,компаньон,0,240525,операции с жильем
7,0,152,50,1,0,m,сотрудник,0,135823,образование
8,2,6929,35,0,1,f,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,1,0,m,сотрудник,0,144425,покупка жилья для семьи


### Категоризация дохода.

Сделаем категоризацию доходов для удобства исследования. Категоризация будет происходит по такому принципу:
* `E` - доход от 0 до 30.000
* `D` - доход от 30.001 до 50.000
* `C` - доход от 50.001 до 200.000
* `B` - доход от 200.001 до 1.000.000
* `A` - доход от 1.000.001

Создадим функцию для этого:

In [33]:
# создаём функцию <income_group>:
# на вход поступает доход в ед.
def income_group(income):
# если доход равен или ниже 30.000, то:
    if income <= 30000:
# присуждается категория 'E'
        return 'E'
# если доход от 30.001 до 50.000, то:
    if income <= 50000:
# присуждается категория 'D'
        return 'D'
# если доход от 50.001 до 200.000, то:
    if income <= 200000:
# присуждается категория 'C'
        return 'C'
# если доход от 200.001 до 1.000.000, то:
    if income <= 1000000:
# присуждается категория 'B'
        return 'B'
# в остальных случаях присуждается категория 'A'
    return 'A'

In [34]:
# создаём столбец 'total_income_category' в таблице df при помощи метода apply() и функции <income_group>:
df['total_income_category'] = df['total_income'].apply(income_group)

Удостоверимся в корректности проведённой категоризации. Выведем первые 10 строк таблицы:

In [35]:
df.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42,0,0,f,сотрудник,0,253875,покупка жилья,B
1,1,4024,36,1,0,f,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623,33,1,0,m,сотрудник,0,145885,покупка жилья,C
3,3,4124,32,1,0,m,сотрудник,0,267628,дополнительное образование,B
4,0,1808,53,1,1,f,пенсионер,0,158616,сыграть свадьбу,C
5,0,926,27,0,1,m,компаньон,0,255763,покупка жилья,B
6,0,2879,43,0,0,f,компаньон,0,240525,операции с жильем,B
7,0,152,50,1,0,m,сотрудник,0,135823,образование,C
8,2,6929,35,0,1,f,сотрудник,0,95856,на проведение свадьбы,C
9,0,2188,41,1,0,m,сотрудник,0,144425,покупка жилья для семьи,C


### Категоризация целей кредита.

Проведём аналогичную категоризацию клиентов, в зависимости от цели кредитования. Разобьём клиентов по группам:
* `операции с автомобилем`
* `операции с недвижимостью`
* `проведение свадьбы`
* `получение образования`

In [36]:
# создадим функцию <purpose_group>:
# на вход функция получает строку с целью кредитования
def purpose_group(purpose):
# если в строке имеется строка 'авто', то
    if 'авто' in purpose:
# присвоить цели категорию 'операции с автомобилем'
        return 'операции с автомобилем'
# если строка содержит строку 'недвижим'
    if (
        ('недвижим' in purpose)
# или
        or
# строку 'жиль', то:
        ('жиль' in purpose)
    ):
# присвоить цели категорию 'операции с недвижимостью'
        return 'операции с недвижимостью'
# если срока содержит строку 'свадь', то:
    if 'свадь' in purpose:
# присвоить цели категорию 'проведение свадьбы'
        return 'проведение свадьбы'
# если строка содержит строку 'образо', то:
    if 'образо' in purpose:
# присвоить цели категорию 'получение образование'
        return 'получение образования'
# в ином случае присвоить цели категорию 'операция неопределена'
    return 'операция неопределена'

In [37]:
# создаёт в датасете новый столбец с категоризацией по типу цели получения кредита:
# применяем метод apply() и функцию <purpose_group>
df['purpose_category'] = df['purpose'].apply(purpose_group)

Удостоверимся в корректности категоризации. Выведем первые 10 строк нового датасета:

In [38]:
df.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437,42,0,0,f,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024,36,1,0,f,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623,33,1,0,m,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124,32,1,0,m,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,1808,53,1,1,f,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,926,27,0,1,m,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,2879,43,0,0,f,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,152,50,1,0,m,сотрудник,0,135823,образование,C,получение образования
8,2,6929,35,0,1,f,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,2188,41,1,0,m,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


***Вывод***

Данные обработаны и готовы к исследованию. В процессе обработки мы столкнулись и устранили:
1. Явные и неявные дубликаты
2. Некорректные и аномальные значения в количественных значениях
3. Неудобный для обработки тип данных
4. Пропуски в количественных значениях

В конечном итоге мы имеет заполненную и хорошо читаемую таблицу. Все количественные пропуски и аномальные значения были заменены медианными значениями, дубликаты полностью устранены. Помимо этого таблица была декомпозирована и были созданы поияснительные датасеты-"словари" для удобства чтения таблицы.

*Основной датасет полностью готов для исследования и ответа на поставленные вопросы.*

## Ответы на вопросы.

### Вопрос 1:

**Есть ли зависимость между количеством детей и возвратом кредита в срок?**

Что бы ответить на этот вопрос, необходимо отследить зависимость между кол-вом детей и процентом невозвратов по кредитам.
Для более удобного отображения информации и вычислений, построим сводную таблицу:

In [39]:
# создаём сводную таблицу с группировкой данных по столбцу 'children' и группировкой значений по столбцу 'debt'
df_pivot_children = df.pivot_table(index='children', columns='debt', values='purpose', aggfunc='count')
# убираем мультииндекс из сформированной сводной таблици
df_pivot_children = df_pivot_children.reset_index()
display(df_pivot_children)

debt,children,0,1
0,0,13142.0,1072.0
1,1,4364.0,444.0
2,2,1858.0,194.0
3,3,303.0,27.0
4,4,37.0,4.0
5,5,9.0,NaN


Как можем заметить - невозвраты по кредитам у клиентов с 5 детьми отсутствуют. Запомним пропуск вручную.

In [40]:
# заполняем пропуск в ячейке
df_pivot_children.loc[5, 1] = 0
# выводим обновлённую таблицу, убеждаемся в отсутствии пропусков
display(df_pivot_children)

debt,children,0,1
0,0,13142.0,1072.0
1,1,4364.0,444.0
2,2,1858.0,194.0
3,3,303.0,27.0
4,4,37.0,4.0
5,5,9.0,0.0


Теперь расчитаем конверсию для каждой группы клиентов. Вычислим общий процент невозвратов в каждой группе:

In [60]:
# создаём новый столбец в таблице для расчёта конверсии
# проводим расчёт отношения невозвратов к общему числу клиентов
# домножаем отношение на 100 для отображения его в процентах
df_pivot_children['debt_conversion'] = (df_pivot_children[1] / (df_pivot_children[0] + df_pivot_children[1])) * 100
display(df_pivot_children)

debt,children,0,1,debt_conversion
0,0,13142.0,1072.0,7.541860
1,1,4364.0,444.0,9.234609
2,2,1858.0,194.0,9.454191
3,3,303.0,27.0,8.181818
4,4,37.0,4.0,9.756098
5,5,9.0,0.0,0.000000


#### Вывод 1:

Как мы можем заметить по столбцу `debt_conversion`, процент невозвратов составил:
* У клиентов без детей `7.5%`
* У клиентов с 1 ребёнком `9.2%`
* У клиентов с 2 детьми `9.4%`
* У клиентов с 3 детьми `8.1%`
* У клиентов с 4 детьми `9.7%`
* У клиентов с 5 детьми `0%`

Можно сделать вывод, что самыми надёжными кредиторами являются клиенты с пятью детьми, однако учитывая кол-во выборки по данной группы, нельзя делать однозначных выводов. Стоит провести дополнительное исследование, которое подтвердит или опровергнет надёжность клиентов данной группы. 
В остальном процент невозвратов коллеблится в пределах одного процента и составляет в среднем `8.5%`

**По предварительным данным исследования можно сказать, что клиенты с пятью детьми - абсолютно надёжные. Надёжность остальных групп клиентов варьируется от `7.5%` до `9.7%`. Поэтому можно сказать, что сильной зависимости по невозврату кредита и кол-ва детей у клиентов не имеется. Однако вопрос нуждается в дополнительном исследование с бОльшей выборкой по ряду групп для более точных данных.**

**Средний процент по невозвратам, не учитывая группу клиентов с 5 детьми - `8.78%`** 

### Вопрос 2:

**Есть ли зависимость между семейным положением и возвратом кредита в срок?**

Для ответа на этот вопрос построим сводную таблицу по аналогичной схеме. При помощи написанного ранее "словаря" семейных статусов, обозначми группы строчными данными, а не числовыми индексами.

In [57]:
# создаём сводную таблицу с группировкой данных по столбцу 'family_status_id' и группировкой значений по столбцу 'debt'
df_pivot_family = df.pivot_table(index='family_status_id', columns='debt', values='purpose', aggfunc='count')
# склеиваем созданную сводную таблицу и словарь семейный статусов
df_pivot_family = df_pivot_family.merge(family_status_dict, on='family_status_id', how='left')
# Приводим таблицу к удобному виду чтения. В конечном варианте - столбец с индексами
# заменён на столбец с наименованиями групп
# заменяем значения индексов на наименования групп
df_pivot_family['family_status_id'] = df_pivot_family['family_status']
# удалем старый столбец с наименованиями групп, новый переименовываем в надлежащих вид
df_pivot_family = df_pivot_family.drop('family_status', 1).rename(columns={
    'family_status_id':'family_status'
})
display(df_pivot_family)

,family_status,0,1
0,женат / замужем,11408,931
1,гражданский брак,3763,388
2,вдовец / вдова,896,63
3,в разводе,1110,85
4,не женат / не замужем,2536,274


Теперь расчитаем конверсию невозвратов в процентах для каждой группы:

In [61]:
# создаём новый столбец debt_conversion для расчета
# отношения невозвратов к общему кол-ву взятых кредитов в процентах
df_pivot_family['debt_conversion'] = ( df_pivot_family[1] / (df_pivot_family[1] + df_pivot_family[0])) * 100
display(df_pivot_family)

,family_status,0,1,debt_conversion
0,женат / замужем,11408,931,7.545182
1,гражданский брак,3763,388,9.347145
2,вдовец / вдова,896,63,6.569343
3,в разводе,1110,85,7.112971
4,не женат / не замужем,2536,274,9.750890


#### Вывод 2:

Как мы можем заметить по столбцу `debt_conversion`, процент невозвратов составил:
* У клиентов в браке `7.5%`
* У клиентов в гражданском браке `9.3%`
* У клиентов вдовцов/вдов `6.5%`
* У клиентов в разводе `7.1%`
* У клиентов не женатых/не замужем `9.7%`

**Таким образом можно сделать выводы, что самыми надёжной группой кредиторов являются овдовевшие клиенты, с процентом невозвратов - `6.5%`. Самой ненадёжной группой клиентов оказались клиенты в гражданском браке - `9.7%`. Учитывая бОльшее колебание в процентах, можно сделать вывод, что семейный статус клиента сильнее влияет на невозврат кредита, чем кол-во детей.**

**Средний процент невозвратов по кредитам составил - `8.02%`**

### Вопрос 3:

**Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

Для ответа на этот вопрос составим, по уже привычной схеме, сводную таблицу.

In [66]:
# создаём сводную таблицу с группировкой данных по столбцу
# 'total_income_category' и группировкой значений по столбцу 'debt'
df_pivot_income = df.pivot_table(index='total_income_category', columns='debt', values='purpose', aggfunc='count')
# убираем мультииндекс из сформированной сводной таблици
df_pivot_income = df_pivot_income.reset_index()
display(df_pivot_income)

debt,total_income_category,0,1
0,A,23,2
1,B,4685,356
2,C,14656,1360
3,D,329,21
4,E,20,2


Так же расчитаем конверсию в процентах для каждой группы:

In [67]:
# создаём новый столбец debt_conversion для расчета
# отношения невозвратов к общему кол-ву взятых кредитов в процентах
df_pivot_income['debt_conversion'] = ( df_pivot_income[1] / (df_pivot_income[1] + df_pivot_income[0])) * 100
display(df_pivot_income)

debt,total_income_category,0,1,debt_conversion
0,A,23,2,8.000000
1,B,4685,356,7.062091
2,C,14656,1360,8.491508
3,D,329,21,6.000000
4,E,20,2,9.090909


#### Вывод 3:

Как мы можем заметить по столбцу `debt_conversion`, процент невозвратов составил:
* У клиентов с уровнем дохода категории A `8%`
* У клиентов с уровнем дохода категории B `7%`
* У клиентов с уровнем дохода категории C `8.49%`
* У клиентов с уровнем дохода категории D `6%`
* У клиентов с уровнем дохода категории E `9%`

Из полученных данных выходит, что клиенты с уровнем дохода категории Е ( доход до 30.000 ) и категории А ( доход от 1.000.001 и выше ) имеют практически один уровень надёжности кредитования, что создаёт впечатление, что данные искажены из-за малого количества выборки клиентов данных категорий. 

**Однако, если учитывать полученные данные в полной мере, то можно сделать вывод, что уровень дохода имеет несколько бОльшее влияние, чем предыдущие факторы. Самыми надёжными кредиторами оказались клиенты с уровнем дохода категории D - `6%` невозвратов по кредитам. Самыми же ненадёжными - клиенты категории C, с `8.49%` невозвратов по кредиту. Однако, как ранее писалось, вопрос требует дополнительного исследования с бОльшей выборкой отдельно взятых категорий для более точных показателей.**

**Средний процент невозвратов по группам - `7.7%`**

### Вопрос 4:

**Как разные цели кредита влияют на его возврат в срок?**

Ответим на этот вопрос по стандартной схеме с помощью сводной таблицы:

In [70]:
# создаём сводную таблицу с группировкой данных по столбцу 'purpose_category' и группировкой значений по столбцу 'debt'
df_pivot_purpose = df.pivot_table(index='purpose_category', columns='debt', values='purpose', aggfunc='count')
# убираем мультииндекс из сформированной сводной таблици
df_pivot_purpose = df_pivot_purpose.reset_index()
display(df_pivot_purpose)

debt,purpose_category,0,1
0,операции с автомобилем,3903,403
1,операции с недвижимостью,10029,782
2,получение образования,3643,370
3,проведение свадьбы,2138,186


Расчитаем процентную конверсию невозвратов по кредитам каждой группы:

In [72]:
# создаём новый столбец debt_conversion для расчета
# отношения невозвратов к общему кол-ву взятых кредитов в процентах
df_pivot_purpose['debt_conversion'] = ( df_pivot_purpose[1] / (df_pivot_purpose[1] + df_pivot_purpose[0])) * 100
display(df_pivot_purpose)

debt,purpose_category,0,1,debt_conversion
0,операции с автомобилем,3903,403,9.359034
1,операции с недвижимостью,10029,782,7.233373
2,получение образования,3643,370,9.220035
3,проведение свадьбы,2138,186,8.003442


#### Вывод 4:

Как мы можем заметить по столбцу `debt_conversion`, процент невозвратов составил:
* У операций с автомобилями `9.35%`
* У операций с недвижимостью `7.23%`
* У операций для получения кредита `9.22%`
* У операций на проведение свадьбы `8%`

**Опираясь на полученные данные, что тип проводимой операции так же имеет низкое влияние на шанс невозврата по кредиту. Самыми надёжными оказались клиенты с операцией по недвижимости - `7.23%`. Самые неблагонадёжные же клиенты с операцией с автомобилей - `9.35%`.**

**Средний процент невозвратов по группам - `8.45%`**

## Общий вывод:

Мы проверили четыре фактора и их влияние на шанс невозврата по кредиту. Из этих исследований можно сделать выводы:
* Отдельно взятый фактор слабо влияет на шанс невозврата, что доказывает колебание среднего процента невозвратов в пределах `1%`;
* Большее влияние на шанс невозврата по кредиту имеет фактор семейного положение клиента и показывает колебание в `3%`.

Однако, стоит учитывать ранее описанную крайне малую выборку по отдельно взятым группам, что можно сказаться на конечном результате. Так же, стоит предположить, что сочетание нескольких факторов может иметь бОльшее влияние, чем отдельно взятый. Обе гипотезы нуждаются в дополнительном исследовании для их подтверждении или опровержении.